In [37]:
import pandas as pd
import numpy as np
from InstructorEmbedding import INSTRUCTOR
import sklearn.cluster

In [49]:
finetuned = pd.read_csv("cv-valid-dev_detection.csv")
finetuned["generated_text"].fillna('', inplace=True)

In [3]:
model = INSTRUCTOR('hkunlp/instructor-large')

.gitattributes: 100%|██████████| 1.48k/1.48k [00:00<?, ?B/s]
2_Dense/config.json: 100%|██████████| 116/116 [00:00<?, ?B/s] 
pytorch_model.bin: 100%|██████████| 3.15M/3.15M [00:00<00:00, 27.0MB/s]
README.md: 100%|██████████| 66.3k/66.3k [00:00<00:00, 33.8MB/s]
config.json: 100%|██████████| 1.53k/1.53k [00:00<00:00, 1.52MB/s]
config_sentence_transformers.json: 100%|██████████| 122/122 [00:00<00:00, 127kB/s]
pytorch_model.bin: 100%|██████████| 1.34G/1.34G [00:21<00:00, 62.9MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 54.1kB/s]
special_tokens_map.json: 100%|██████████| 2.20k/2.20k [00:00<?, ?B/s]
spiece.model: 100%|██████████| 792k/792k [00:00<00:00, 35.1MB/s]
tokenizer.json: 100%|██████████| 2.42M/2.42M [00:00<00:00, 9.44MB/s]
tokenizer_config.json: 100%|██████████| 2.41k/2.41k [00:00<?, ?B/s]
modules.json: 100%|██████████| 461/461 [00:00<00:00, 462kB/s]


load INSTRUCTOR_Transformer
max_seq_length  512


In [53]:
instruction = '''Represent the similarity of the words in the sentence to the phrase "be careful" for clustering: '''
sentences = [[instruction,'I love life'],
            #  [instruction,'destroy'],
             [instruction,'I like ice cream'],
             [instruction,'Today was a sunny day'],
             [instruction,"Beware"],
             [instruction,'The bomb was about to destruct'],
             [instruction,'watch out'],
             [instruction,'be careful with your prognostications said the stranger']
             ]
embeddings = model.encode(sentences)
clustering_model = sklearn.cluster.KMeans(n_clusters=2)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment)

[0 0 0 1 1 1 1]


c:\Users\calic\Documents\GitHub\htx-tech-test\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [41]:
instruction = '''Represent the similarity of the words in the sentence to the word "destroy" for clustering: '''
model_inputs = [[instruction, x.lower()] for x in finetuned["generated_text"].values]
model_inputs.append([instruction,'destroy'])
embeddings = model.encode(model_inputs)
clustering_model = sklearn.cluster.KMeans(n_clusters=2)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment[-1])

c:\Users\calic\Documents\GitHub\htx-tech-test\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0


In [42]:
finetuned['destroy_similar'] = np.where(cluster_assignment[:-1] == 0, True, False)

In [45]:
instruction = '''Represent the similarity of the words in the sentence to the word "stranger" for clustering: '''
model_inputs = [[instruction, x.lower()] for x in finetuned["generated_text"].values]
model_inputs.append([instruction,'stranger'])
embeddings = model.encode(model_inputs)
clustering_model = sklearn.cluster.KMeans(n_clusters=2)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment[-1])

c:\Users\calic\Documents\GitHub\htx-tech-test\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


1


In [46]:
finetuned['stranger_similar'] = np.where(cluster_assignment[:-1] == 1, True, False)

In [54]:
instruction = '''Represent the similarity of the words in the sentence to the phrase "be careful" for clustering: '''
model_inputs = [[instruction, x.lower()] for x in finetuned["generated_text"].values]
model_inputs.append([instruction,'be careful'])
embeddings = model.encode(model_inputs)
clustering_model = sklearn.cluster.KMeans(n_clusters=2)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment[-1])

c:\Users\calic\Documents\GitHub\htx-tech-test\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


1


In [55]:
finetuned['be_careful_similar'] = np.where(cluster_assignment[:-1] == 1, True, False)

In [60]:
finetuned["similarity"] = finetuned[["destroy_similar","stranger_similar","be_careful_similar"]].any(bool_only=True, axis=1)

In [62]:
finetuned["similarity"].sum()

2929

In [63]:
finetuned.to_csv('cv-valid-dev_detection.csv')